# 4.3.2 有価証券報告書内のMD＆Aが記載されているhtmファイルを抽出　

　Zipファイルで送られてきた大量の有価証券書のデータをPythonにより自動的に解凍した。その後、有価証券報告書内のMD＆A情報のみが記載されたhtmファイルのみを抽出をした。

In [155]:
import pandas as pd
import glob
import os
import zipfile
from datetime import datetime as dt
import bs4 
import re

In [156]:
def make_industry_zip_list_hash(data_frame, data_frame_name):
    
    industry_zip_list_hash = {}
    industry_li = make_type_of_industry_list(data_frame)
    zip_files = call_zip_files(data_frame_name)
    for industry in industry_li :
        industry_zip_list_hash[industry]= list(filter(lambda x: industry in x , zip_files))
    
    return   industry_zip_list_hash
    

def make_type_of_industry_list(data_frame : pd.DataFrame, industry_col="[業種（東証）]"):
    return list(set(data_frame[industry_col]))


def call_zip_files(data_frame_name):
    zip_files = glob.glob(f"../**/SampleData/{data_frame_name}/**/**.zip", recursive=True)
    return zip_files

def call_unziped_htm_files_dir(data_frame_name: str):

    unziped_htm_files_dir = os.getcwd()+  "/UnzipedHtmFiles"
    if not os.path.exists(unziped_htm_files_dir) :
        os.mkdir(unziped_htm_files_dir)

    unziped_htm_files_dir_with_df_name = unziped_htm_files_dir + f"/{data_frame_name}"
    if not os.path.exists(unziped_htm_files_dir_with_df_name) :
        os.mkdir(unziped_htm_files_dir_with_df_name)
    
    return unziped_htm_files_dir_with_df_name


#--------------------------------------------------------------------
def unzip_html_to_unziped_htm_files_dir(industry_zip_list_hash, filepath_unziped):
    
    sum_files_len = sum(map(len, industry_zip_list_hash.values()))
    zip_files_len = 1
    
    for industy_name, zip_files in industry_zip_list_hash.items() :
        zip_files_len += len(zip_files)
        
        industry_dir = call_industry_dir(filepath_unziped, industy_name)
    
        for idx ,zip_file in enumerate(zip_files):

            count = 0
            start_idx = 2
            while count < 2 :
                try:
                    with zipfile.ZipFile(zip_file) as existing_zip:

                        required_file = existing_zip.namelist()[start_idx]
                        date_str = required_file[-20:-10]
                        date_dt = dt.strptime(date_str, '%Y-%m-%d')

                        existing_zip.extract(required_file, industry_dir)

                        print(f"{idx + 1} / {len(zip_files)} ||  {zip_files_len - 1}/{sum_files_len}")
                except Exception :
                    print(zip_file)
                    with open("unzipping_html_error.txt", "a") as f:
                        f.write(zip_file + "\n")
                
                htm_path = glob.glob(industry_dir + "/"  + required_file, recursive=False)[0]
                number_of_chapters = count_the_number_of_chapters(htm_path)
                count += number_of_chapters
                print(f"number_of_chapters: {number_of_chapters}")
                start_idx += 1
                
                
def call_industry_dir(filepath_unziped, industry_name: str):
    industry_dir =  filepath_unziped+  f"/{industry_name}"
    if not os.path.exists(industry_dir) :
        os.mkdir(industry_dir)
    return industry_dir


def count_the_number_of_chapters(htm_path , chapter_numbers=['１', '1','３', '3']) :
    
    with open(htm_path, mode="r") as f:
        htm = f.read()
        sp = bs4.BeautifulSoup(htm, "html.parser")

    htm_contents = sp.find('body').div.contents
    
    #MD＆A情報を持つ章の個数
    count = 0
    
    for content in htm_contents :
        #bs4.element.Tag型ではないとfindメソッドでtypeエラーを起こすため判定
        if not isinstance(content, bs4.element.Tag) :
            continue
        
        found = content.find(class_=re.compile("smt_head"))    
        if not  found:
            found = content.find("h3")
            if not  found:
                continue
    
        title = found.text
        #md_aの章番号を指定
            # "1【経営方針、経営環境及び対処すべき課題等】",
            # "3【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】"
        #企業によって全角と半角か別れる可能性があるためどちらも判定
        if title[0] in  chapter_numbers :
              count += 1
            
    return count

In [157]:
print("dfd")

dfd


In [158]:
f = call_zip_files("renketsu") + call_zip_files("hirenketsu")
len_f = len(f)
print(len_f,  len_f/3)

6840 2280.0


In [159]:
#renkestsuとhirenketsuを設定
data_frame_name ="renketsu" 
data_frame = pd.read_csv("/home/jovyan/1CalliingEdinetApi"+f"/EdinetIdxFiles/edinet_{data_frame_name}.csv", skiprows=4)

In [ ]:
industry_zip_list_hash = make_industry_zip_list_hash(data_frame, data_frame_name)
filepath_unziped = call_unziped_htm_files_dir(data_frame_name)
unzip_html_to_unziped_htm_files_dir(industry_zip_list_hash, filepath_unziped)

1 / 237 ||  237/6171
number_of_chapters: 2
2 / 237 ||  237/6171
number_of_chapters: 2
3 / 237 ||  237/6171
number_of_chapters: 2
4 / 237 ||  237/6171
number_of_chapters: 2
5 / 237 ||  237/6171
number_of_chapters: 2
6 / 237 ||  237/6171
number_of_chapters: 2
7 / 237 ||  237/6171
number_of_chapters: 2
8 / 237 ||  237/6171
number_of_chapters: 2
9 / 237 ||  237/6171
number_of_chapters: 2
10 / 237 ||  237/6171
number_of_chapters: 2
11 / 237 ||  237/6171
number_of_chapters: 2
12 / 237 ||  237/6171
number_of_chapters: 2
13 / 237 ||  237/6171
number_of_chapters: 2
14 / 237 ||  237/6171
number_of_chapters: 2
15 / 237 ||  237/6171
number_of_chapters: 2
16 / 237 ||  237/6171
number_of_chapters: 2
17 / 237 ||  237/6171
number_of_chapters: 2
18 / 237 ||  237/6171
number_of_chapters: 2
19 / 237 ||  237/6171
number_of_chapters: 2
20 / 237 ||  237/6171
number_of_chapters: 2
21 / 237 ||  237/6171
number_of_chapters: 2
22 / 237 ||  237/6171
number_of_chapters: 2
23 / 237 ||  237/6171
number_of_chapters:

number_of_chapters: 2
187 / 237 ||  237/6171
number_of_chapters: 2
188 / 237 ||  237/6171
number_of_chapters: 2
189 / 237 ||  237/6171
number_of_chapters: 2
190 / 237 ||  237/6171
number_of_chapters: 2
191 / 237 ||  237/6171
number_of_chapters: 2
192 / 237 ||  237/6171
number_of_chapters: 2
193 / 237 ||  237/6171
number_of_chapters: 2
194 / 237 ||  237/6171
number_of_chapters: 2
195 / 237 ||  237/6171
number_of_chapters: 2
196 / 237 ||  237/6171
number_of_chapters: 2
197 / 237 ||  237/6171
number_of_chapters: 2
198 / 237 ||  237/6171
number_of_chapters: 2
199 / 237 ||  237/6171
number_of_chapters: 2
200 / 237 ||  237/6171
number_of_chapters: 2
201 / 237 ||  237/6171
number_of_chapters: 2
202 / 237 ||  237/6171
number_of_chapters: 2
203 / 237 ||  237/6171
number_of_chapters: 2
204 / 237 ||  237/6171
number_of_chapters: 2
205 / 237 ||  237/6171
number_of_chapters: 2
206 / 237 ||  237/6171
number_of_chapters: 2
207 / 237 ||  237/6171
number_of_chapters: 2
208 / 237 ||  237/6171
number_of_